In [2]:
import numpy as np
import random
import math
from operator import mul,neg

from EstrategiaParticionado import ValidacionCruzada
from datos import Datos
from Clasificador import Clasificador

from scipy.special import expit
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split


vector = np.array([1,17.99,10.38,122.8,1001,0.1184,0.2776,0.3001,0.1471,0.2419,0.07871,1.095,0.9053,8.589,153.4,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.6,2019,0.1622,0.6656,0.7119,0.2654,0.4601,0.1189])


In [3]:
def producto_escalar(w,x):
    return sum(map(mul,w,x))


In [4]:
def calcular_sigmoidal (a):
    return expit(a)

In [5]:
def generar_vector(d):
    return np.array([ random.uniform(-1,1) for i in range(d) ])


In [43]:
def generar_vector_datos(dato):
    return np.insert(dato,0,1)

In [54]:
generar_vector_datos(np.array([0,8,3,2]))

array([1, 0, 8, 3, 2])

In [6]:
pe = producto_escalar(generar_vector(len(vector)),vector)

In [55]:
class ClasificadorRegresionLogistica(Clasificador):
    w = np.array([])
    n = 0.0 #cte aprendizaje
    num_epocas = 0

    def __init__(self,num_epocas, n):
        super(ClasificadorRegresionLogistica, self).__init__()
        self.num_epocas = num_epocas
        self.n = n
    def __generar_vector_nuevo(self,dato,sigmoidal,clase):
        factor = self.n * (sigmoidal - clase)
        return self.w - np.array(list(map(lambda x : x * factor, dato)))
    def entrenamiento(self, datosTrain, atributosDiscretos, diccionario):
        self.w = generar_vector(len(atributosDiscretos))
        for epoca in range(self.num_epocas):
            for dato in datosTrain:
                vector_datos = generar_vector_datos(dato[:-1])
                pe = producto_escalar( self.w,vector_datos)
                sigmoidal = calcular_sigmoidal(pe)
                self.w = self.__generar_vector_nuevo(vector_datos,
                                                     sigmoidal,
                                                     dato[-1])
        return self.w
    def __clasifica_dato(self,dato_test):
        return int(calcular_sigmoidal(producto_escalar(self.w,dato_test)) > 0.5)
    def clasifica (self, datosTest, atributosDiscretos, diccionario):
        return list(map(self.__clasifica_dato, datosTest))


In [56]:
wdbc = Datos('../ficheros/ConjuntosDatos/wdbc.data',True)
example3 = Datos('../ficheros/ConjuntosDatos/example3.data',True)
example4 = Datos('../ficheros/ConjuntosDatos/example4.data',True)

In [87]:
estrategia = ValidacionCruzada(10)
sklearn_cv = KFold(n_splits=10,shuffle=True)
clasificador_sklearn = LogisticRegression(n_jobs=2) 

In [47]:
#clasificador = ClasificadorRegresionLogistica(10,1)

scores = cross_val_score(clasificador_sklearn, wdbc.datos[:,:-1],wdbc.datos[:,-1],cv = sklearn_cv)
print(np.mean(scores))

0.597368421053


In [131]:
X_train,X_test,y_train,y_test = train_test_split(wdbc.datos[:,:-1],
                                                 wdbc.datos[:,-1], 
                                                 test_size = 0.2)
clasificador_sklearn.fit(X_train,y_train)
print(clasificador_sklearn.score(X_test,y_test))

0.570175438596


In [132]:
print(len(wdbc.datos[0]))

31


In [153]:
X_train,X_test,y_train,y_test = train_test_split(example3.datos[:,:-1],
                                                 example3.datos[:,-1], 
                                                 test_size = 0.2)
clasificador_sklearn.fit(X_train,y_train)
print(clasificador_sklearn.score(X_test,y_test))

0.5


In [154]:
clasificador = ClasificadorRegresionLogistica(100,1)

In [155]:
errores = clasificador.validacion(estrategia,wdbc,clasificador)

print(1 - np.mean(errores))

0.510934065934


In [51]:
errores = clasificador.validacion(estrategia,example3,clasificador)

print(1 - np.mean(errores))

0.4975


In [52]:
errores = clasificador.validacion(estrategia,example4,clasificador)

print(1 - np.mean(errores))

0.52
